In [1]:
import boto3

In [2]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/serve.py /opt/ml/code/serve.py

RUN pip3 install -r /opt/ml/requirements.txt
RUN pip3 install flask gunicorn

ENV SAGEMAKER_PROGRAM serve.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/serve.py"]

Overwriting Dockerfile


In [3]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'prophet_infernece'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  310.3kB
Step 1/8 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/8 : COPY requirements.txt /opt/ml/requirements.txt
 ---> Using cache
 ---> 7f1423ae1d9f
Step 3/8 : COPY code/serve.py /opt/ml/code/serve.py
 ---> a0658d27aca6
Step 4/8 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Running in e66fd563c734
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 85.5 MB/s eta 0:00

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [ ]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [ ]:
!docker push $processing_repository_uri